# Practica 1: Spark
***
Rodriguez Nuñez Diego Eduardo

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

## Paso 1
Descargar los datos de https://github.com/databricks/Spark-The-DefinitiveGuide/tree/master, leer los datos de los vuelos registrados en el archivo 2015-summary.csv y ordena los datos de acuerdo con la columna count, además imprime el plan de como se ejecuta esto en el cluster.


In [2]:
# Creamos la sesion de Spark
spark = SparkSession.builder.appName("Flight Analysis").getOrCreate()

In [3]:
# Cargamos lo datos
df = spark.read.option("header", "true").csv("2015-summary.csv",inferSchema=True)

In [4]:
# Ordenamos los datos por el numero de vuelos
df = df.orderBy(desc("count"))

In [5]:
# Mostramos el plan de ejecucion
df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#19 DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(count#19 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=27]
      +- FileScan csv [DEST_COUNTRY_NAME#17,ORIGIN_COUNTRY_NAME#18,count#19] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/c:/Users/dern9/Desktop/Big-Data/Practica 1/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




## Paso 2
Encuentra el máximo numero de vuelos desde y hacia cualquier ubicación determinada. 

In [6]:
# Agrupamos lo datos por origen y destino
df_origin = df.groupBy("ORIGIN_COUNTRY_NAME").sum("count")
df_dest = df.groupBy("DEST_COUNTRY_NAME").sum("count")

# Unimos los dataframes
df_total = df_origin.union(df_dest)

# Encontramos el total de vuelos por pais
df_max_locations = df_total.groupBy("ORIGIN_COUNTRY_NAME").sum("sum(count)").orderBy(desc("sum(sum(count))")).withColumnRenamed("sum(sum(count))", "total_flights")

# Mostramos los resultados
df_max_locations.show()

+-------------------+-------------+
|ORIGIN_COUNTRY_NAME|total_flights|
+-------------------+-------------+
|      United States|       823318|
|             Canada|        16882|
|             Mexico|        14327|
|     United Kingdom|         3995|
|              Japan|         3044|
|            Germany|         2804|
| Dominican Republic|         2773|
|        The Bahamas|         1941|
|             France|         1887|
|        South Korea|         1875|
|           Colombia|         1740|
|              China|         1692|
|             Brazil|         1472|
|        Netherlands|         1436|
|            Jamaica|         1378|
|         Costa Rica|         1196|
|        El Salvador|         1069|
|             Panama|          975|
|               Cuba|          944|
|              Spain|          862|
+-------------------+-------------+
only showing top 20 rows



## Paso 3
Encuentra los top 5 destinos (países) en los datos y muéstralo en la terminal


In [7]:
df_top_5 = df.groupBy("DEST_COUNTRY_NAME").sum("count").orderBy(desc("sum(count)"))
df_top_5.show(5)

+-----------------+----------+
|DEST_COUNTRY_NAME|sum(count)|
+-----------------+----------+
|    United States|    411352|
|           Canada|      8399|
|           Mexico|      7140|
|   United Kingdom|      2025|
|            Japan|      1548|
+-----------------+----------+
only showing top 5 rows



## Consulta

Contar los vuelos que tienen el mismo origen y destino

In [9]:
df_consulta = df.filter(df.DEST_COUNTRY_NAME == df.ORIGIN_COUNTRY_NAME).groupBy("DEST_COUNTRY_NAME").sum("count").orderBy(desc("sum(count)"))
df_consulta.show()

+-----------------+----------+
|DEST_COUNTRY_NAME|sum(count)|
+-----------------+----------+
|    United States|    370002|
+-----------------+----------+

